In [1]:
import pandas as pd
import re
import spacy
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression
import numpy as np
from sklearn.utils.class_weight import compute_class_weight
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer
from imblearn.over_sampling import ADASYN


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [59]:
tam_df=pd.read_csv("/content/drive/MyDrive/sentiment analysis/Tam-SA-train.csv")
tam_dev=pd.read_csv("/content/drive/MyDrive/sentiment analysis/Tam-SA-val.csv")
tam_test=pd.read_csv("/content/drive/MyDrive/sentiment analysis/Tam-SA-test-without-labels.csv")

In [60]:
tam_df = pd.concat([tam_df, tam_dev], ignore_index=True)

In [62]:
#tam_df
for i in range(len(tam_df)):
  curr_text=tam_df.loc[i]["Text"].strip()
  curr_text=re.sub(r'\s+',' ',curr_text)
  tam_df.at[i,"Text"]=curr_text
tam_df


,Text,Label
0,Ennq pa idhu paei padama twist nalla irkkae,Positive
1,Na oru thalaivar veriyan....intha padam pakanu...,unknown_state
2,last shot apdiye moratu kaala paatha feeling,Positive
3,Darbar motion poster see after petta trailer,unknown_state
4,"ln 9 minutes 88k likes , Ajith sir fans like h...",Positive
...,...,...
34960,Thalaaaaaaaaa love u sirr frm andhra pradesh,Positive
34961,BGM sounds like Arjun Reddy Bgm.! ?,Positive
34962,sethupathi manirathnam Aravind swami arun Vija...,Positive
34963,"சோழிய வேளாளர், சோழ வேளாளர், சூரிய குல சத்திரியர்.",Positive


In [63]:
tam_df = tam_df[~tam_df['Text'].str.strip().eq('')]
tam_df

,Text,Label
0,Ennq pa idhu paei padama twist nalla irkkae,Positive
1,Na oru thalaivar veriyan....intha padam pakanu...,unknown_state
2,last shot apdiye moratu kaala paatha feeling,Positive
3,Darbar motion poster see after petta trailer,unknown_state
4,"ln 9 minutes 88k likes , Ajith sir fans like h...",Positive
...,...,...
34960,Thalaaaaaaaaa love u sirr frm andhra pradesh,Positive
34961,BGM sounds like Arjun Reddy Bgm.! ?,Positive
34962,sethupathi manirathnam Aravind swami arun Vija...,Positive
34963,"சோழிய வேளாளர், சோழ வேளாளர், சூரிய குல சத்திரியர்.",Positive


In [64]:

# Tamil stopwords (Tamil script and transliterated English)
tamil_stopwords_tamil_script = [
    "அது", "ஆனால்", "என்று", "அவர்", "அவர்கள்", "அவள்", "இங்கு", "இல்லை",
    "என்ன", "எப்போது", "எந்த", "எது", "ஓவ்வொரு", "தான்", "உனக்கு", "உன்னை",
    "எனது", "எனக்கு", "உன்", "நான்", "நாம்", "நீங்கள்", "உங்களுக்கு", "அவர்களுக்கு",
    "இதை", "அதை", "இது", "அந்த", "இவை", "அவை", "இந்த", "அங்கே", "இங்கே",
    "எதற்கு", "எதை", "எப்பொழுது", "எவ்வாறு", "யார்", "யாருடைய", "யாருக்கு",
    "எவர்", "எவரும்", "இருக்கும்", "இருந்தது", "இருந்தான்", "இருக்கின்றன",
    "இருந்தாள்", "மற்றும்", "மேலும்", "அல்லது", "என்பதால்", "கூட", "மட்டும்",
    "அதனால்", "அதில்", "இதில்", "எனவே","நீ"
]

tamil_stopwords_transliterated = [
    "athu", "aanal", "endru", "avar", "avargal", "aval", "ingu", "illai",
    "enna", "eppodhu", "entha", "ethu", "ovvoru", "thaan", "unakku", "unnai",
    "enathu", "enakku", "un", "naan","na", "naam", "neengal", "ungalukku", "avargalukku",
    "ithai", "athai", "idhu", "andha", "ivai","ah","nu","ippa","than","en","un", "avai", "indha","ahna","intha", "ingae", "angae",
    "edharku", "edhai", "eppadi", "eppozhuthu", "evvaru", "yaar", "yaarudaiya",
    "yaarukku", "evar", "evarum", "irukkum", "irundhathu", "irundhaan", "irukkinrana",
    "irundhaal", "matrum", "melum", "allathu", "enbadhal", "kooda", "mattum", "la",
    "athan", "athil", "ithan", "ithil", "enavae"
]

# English stopwords (a common list of stopwords in English)
english_stopwords = [
    "i", "me", "my", "myself", "we", "our", "ours", "ourselves", "you", "your", "yours",
    "yourself", "yourselves", "he", "him", "his", "himself", "she", "her", "hers",
    "herself", "it", "its", "itself", "they", "them", "their", "theirs", "themselves",
    "what", "which", "who", "whom", "this", "that", "these", "those", "am", "is", "are",
    "was", "were", "be", "been", "being", "have", "has", "had", "having", "do", "does",
    "did", "doing", "a", "an", "the", "and", "but", "if", "or", "because", "as", "until",
    "while", "of", "at", "by", "for", "with", "about", "against", "between", "into",
    "through", "during", "before", "after", "above", "below", "to", "from", "up", "down",
    "in", "out", "on", "off", "over", "under", "again", "further", "then", "once",
    "here", "there", "when", "where", "why", "how", "all", "any", "both", "each", "few",
    "more", "most", "other", "some", "such", "no", "nor", "not", "only", "own", "same",
    "so", "than", "too", "very", "s", "t", "can", "will", "just", "don", "should", "now",
    "d", "ll", "m", "o", "re", "ve", "y", "ain", "aren", "couldn", "didn", "doesn",
    "hadn", "hasn", "haven", "isn", "ma", "mightn", "mustn", "needn", "shan", "shouldn",
    "wasn", "weren", "won", "wouldn"
]

def is_tamil_word(word):
    return bool(re.search(r'[\u0B80-\u0BFF]', word))  # Tamil Unicode block

def remove_stopwords(text, tamil_stopwords_script, tamil_stopwords_trans, english_stopwords):
    words = text.split()
    cleaned_words = []

    for word in words:
        word_lower = word.lower()

        if is_tamil_word(word):
            if word not in tamil_stopwords_script:
                cleaned_words.append(word)
        elif word_lower not in tamil_stopwords_trans and word_lower not in english_stopwords:
            cleaned_words.append(word)

    return ' '.join(cleaned_words)

for i in range(len(tam_df)):
  text=tam_df.loc[i]["Text"]
  cleaned_text = remove_stopwords(text,tamil_stopwords_tamil_script, tamil_stopwords_transliterated, english_stopwords)
  tam_df.loc[i,"Text"]=cleaned_text
for i in range(len(tam_test)):
  text=tam_test.loc[i]["Text"]
  cleaned_text = remove_stopwords(text,tamil_stopwords_tamil_script, tamil_stopwords_transliterated, english_stopwords)
  tam_test.loc[i,"Text"]=cleaned_text





In [65]:
def extract_text_features(text):


    timestamp_pattern = r'\b\d{1,2}:\d{2}\b'
    number_format_pattern = r'\b\d+[KMkm]\b'
    plain_numbers_pattern = r'\b\d+\b'

    # Apply patterns in the correct order
    text = re.sub(timestamp_pattern, '', text)  # Remove timestamps first
    text = re.sub(number_format_pattern, '', text)  # Then remove K/M suffix-based numbers
    text = re.sub(plain_numbers_pattern, '', text)  # Finally, remove plain numbers
    text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation

    # Return features as a dictionary and the cleaned text
    return {
        'cleaned_text': text.strip()
    }


In [66]:
# Process `tam_df`
import string
for i in range(len(tam_df)):
    curr_text = tam_df.loc[i, "Text"].strip()
    curr_text = re.sub(r'\s+', ' ', curr_text)  # Normalize spaces
    features = extract_text_features(curr_text)

    # Add features to the DataFrame
    tam_df.at[i, "Text"] = features['cleaned_text']  # Update cleaned text
    for feature, value in features.items():
        if feature != 'cleaned_text':
            tam_df.at[i, feature] = value

# Process `tam_dev`
for i in range(len(tam_test)):
    curr_text = tam_test.loc[i, "Text"].strip()
    curr_text = re.sub(r'\s+', ' ', curr_text)  # Normalize spaces
    features = extract_text_features(curr_text)

    # Add features to the DataFrame
    tam_test.at[i, "Text"] = features['cleaned_text']  # Update cleaned text
    for feature, value in features.items():
        if feature != 'cleaned_text':
            tam_test.at[i, feature] = value


In [70]:
for i in range(len(tam_test)):
  curr_text=tam_dev.loc[i]["Text"].lower()
  tam_test.at[i,"Text"]=curr_text
tam_test
for i in range(len(tam_df)):
  curr_text=tam_df.loc[i]["Text"].lower()
  tam_df.at[i,"Text"]=curr_text
tam_test

,Id,Text
0,SA_Ta_01,just 2k likes to reach 400k likes​
1,SA_Ta_02,massssssssssssssssssss overload....... sema v...
2,SA_Ta_03,she's looks like a dummy (mom ki gudiya)
3,SA_Ta_04,எந்த ட்ரைலர் சிறந்தது பிசாசு விருப்பம் சைக்க...
4,SA_Ta_05,adichi under wear oda oda vituruvean lol
...,...,...
3454,SA_Ta_3455,jee exact ah enakum indha noi irukuji funeral ...
3455,SA_Ta_3456,please kaappaanla nadippin nayagan suriya nu ...
3456,SA_Ta_3457,pakka paisa vasol movie... mass movie after na...
3457,SA_Ta_3458,waiting to seeeeeeeeee anna mass


In [71]:
import re

def remove_duplicate_words(text):
    words = text.split()  # Split text into words
    seen = set()  # Set to track seen words
    unique_words = []  # List to store unique words

    for word in words:
        if word not in seen:  # Check if word is not a duplicate
            seen.add(word)  # Add word to seen set
            unique_words.append(word)  # Add to the unique words list

    return ' '.join(unique_words)  # Reconstruct the text

# Example usage on a DataFrame
for i in range(len(tam_df)):
    tam_df.at[i, "Text"] = remove_duplicate_words(tam_df.at[i, "Text"])
for i in range(len(tam_test)):
    tam_test.at[i,"Text"]=remove_duplicate_words(tam_test.at[i,"Text"])


In [73]:
tam_df = tam_df.drop_duplicates(subset=["Text"], keep='first').reset_index(drop=True)


In [72]:
tam_test=tam_test.drop_duplicates(subset=["Text"], keep='first').reset_index(drop=True)

In [77]:
tam_dev=tam_test

,Id,Text
0,SA_Ta_01,just 2k likes to reach 400k likes​
1,SA_Ta_02,massssssssssssssssssss overload....... sema ve...
2,SA_Ta_03,she's looks like a dummy (mom ki gudiya)
3,SA_Ta_04,எந்த ட்ரைலர் சிறந்தது பிசாசு விருப்பம் சைக்கோ ...
4,SA_Ta_05,adichi under wear oda vituruvean lol
...,...,...
3447,SA_Ta_3455,jee exact ah enakum indha noi irukuji funeral ...
3448,SA_Ta_3456,please kaappaanla nadippin nayagan suriya nu p...
3449,SA_Ta_3457,pakka paisa vasol movie... mass movie after na...
3450,SA_Ta_3458,waiting to seeeeeeeeee anna mass


In [78]:
def remove_repeating_last_letters(text):
    words = text.split()  # Split the text into words
    updated_words = []
    for word in words:
        while len(word) > 1 and word[-2] == word[-1]:  # Check for repeating last letters
            word = word[:-1]  # Remove the last letter
        updated_words.append(word)
    return ' '.join(updated_words)  # Join the words back into a string

# Process tam_df
for i in range(len(tam_df)):
    curr_text = tam_df.loc[i]["Text"]
    tam_df.at[i, "Text"] = remove_repeating_last_letters(curr_text)

# Process tam_dev
for i in range(len(tam_dev)):
    curr_text = tam_dev.loc[i]["Text"]
    tam_dev.at[i, "Text"] = remove_repeating_last_letters(curr_text)


In [80]:
from sklearn.preprocessing import LabelEncoder

# Initialize the LabelEncoder
label_encoder = LabelEncoder()
tam_df['classes']=0
# Apply label encoding to the 'labels' column and create a new 'classes' column
tam_df['classes'] = label_encoder.fit_transform(tam_df['Label'])

# Check the DataFrame to see the new 'classes' column
print(tam_df[['Label', 'classes']])

               Label  classes
0           Positive        2
1      unknown_state        3
2           Positive        2
3      unknown_state        3
4           Positive        2
...              ...      ...
34366       Positive        2
34367       Positive        2
34368       Positive        2
34369       Positive        2
34370  unknown_state        3

[34371 rows x 2 columns]


In [81]:
# Check unique values in the encoded column
print(tam_df['classes'].unique())

# If you have access to the original dataset, compare it
original_labels = ['negative', 'mixed_feelings', 'positive','unknown_state']  # Replace with your dataset's original labels
encoded_labels = sorted(tam_df['classes'].unique())  # Assumes ascending order mapping
mapping = dict(zip(original_labels, encoded_labels))
print("Mapping:", mapping)


[2 3 1 0]
Mapping: {'negative': 0, 'mixed_feelings': 1, 'positive': 2, 'unknown_state': 3}


In [82]:
import re

def remove_numbers(text):
    # Regex to match any number (with optional suffixes like 's', 'th', etc.)
    text = re.sub(r'\b\d+\w*\b', '', text)
    return text.strip()

# Example usage on a DataFrame
for i in range(len(tam_df)):
    tam_df.at[i, "Text"] = remove_numbers(tam_df.at[i, "Text"])
for i in range(len(tam_dev)):
    tam_dev.at[i, "Text"] = remove_numbers(tam_dev.at[i, "Text"])


In [83]:

def remove_numerical_words(text):
    # List of numerical terms to remove
    numerical_words = r'\b(million|minutes|seconds|hours|secs|mins|hrs|billion|trillion|thousand|hundred|k|m|bn|tn)\b'
    # Use regex to replace them with an empty string
    text = re.sub(numerical_words, '', text, flags=re.IGNORECASE)
    return text.strip()


for i in range(len(tam_df)):
    tam_df.at[i, "Text"] = remove_numerical_words(tam_df.at[i, "Text"])
for i in range(len(tam_dev)):
    tam_dev.at[i, "Text"] = remove_numerical_words(tam_dev.at[i, "Text"])


In [85]:
tam_train_df = pd.DataFrame(tam_df)
vectorizer = TfidfVectorizer(stop_words='english',ngram_range=(1, 2), max_features=5000)
X = vectorizer.fit_transform(tam_df['Text'])
y = tam_df['classes']

# Train-Test Split
X_train = vectorizer.fit_transform(tam_df['Text'])
y_train = tam_df['classes']

X_dev = vectorizer.transform(tam_dev['Text'])
X_resampled=X_train
y_resampled=y_train

In [86]:
from imblearn.over_sampling import SMOTE

# Initialize SMOTE for oversampling the minority class
smote = SMOTE(random_state=42)

# Apply SMOTE to the training data (X_train and y_train)
X_resampled, y_resampled = smote.fit_resample(X_resampled, y_train)

# Check the shape of the resampled data
print(f"Resampled X_train shape: {X_resampled.shape}")
print(f"Resampled y_train shape: {y_resampled.shape}")


Resampled X_train shape: (80296, 5000)
Resampled y_train shape: (80296,)


In [87]:
print(f"X_resampled shape: {X_resampled.shape}")
print(f"y_train shape: {y_train.shape}")


X_resampled shape: (80296, 5000)
y_train shape: (34371,)


In [88]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

model = LogisticRegression(max_iter=30000,C=0.3,solver='liblinear')
model.fit(X_resampled, y_resampled)

# Make predictions
y_pred = model.predict(X_dev)
tam_dev["Labels"]=y_pred


In [90]:
tam_test=tam_dev

In [91]:
for i in range(len(tam_test)):
  if tam_test.loc[i,'Labels']==3:
    tam_test.loc[i,'Labels']='unknown_state'
  elif tam_test.loc[i,'Labels']==2:
    tam_test.loc[i,'Labels']='positive'
  elif tam_test.loc[i,'Labels']==1:
    tam_test.loc[i,'Labels']='mixed_feelings'
  else:
    tam_test.loc[i,'Labels']='negative'
tam_test

<ipython-input-91-36c84696fba4>:3: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'unknown_state' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  tam_test.loc[i,'Labels']='unknown_state'


,Id,Text,Labels
0,SA_Ta_01,just likes to reach likes​,unknown_state
1,SA_Ta_02,mas overload. sema vera level,positive
2,SA_Ta_03,she's looks like a dummy (mom ki gudiya),negative
3,SA_Ta_04,எந்த ட்ரைலர் சிறந்தது பிசாசு விருப்பம் சைக்கோ ...,negative
4,SA_Ta_05,adichi under wear oda vituruvean lol,negative
...,...,...,...
3447,SA_Ta_3455,je exact ah enakum indha noi irukuji funeral h...,mixed_feelings
3448,SA_Ta_3456,please kaappaanla nadippin nayagan suriya nu p...,unknown_state
3449,SA_Ta_3457,pakka paisa vasol movie. mas movie after naras...,positive
3450,SA_Ta_3458,waiting to se anna mas,positive


In [92]:
tam_dev= tam_dev.drop(columns=['Text'])


In [93]:
tam_test.to_csv("/content/drive/MyDrive/sentiment analysis/TensorTalk_Tamil_run1.tsv", index=False)

In [95]:
model = MultinomialNB(alpha=0.01)
model.fit(X_resampled, y_resampled)

# Predictions
y_pred = model.predict(X_dev)

tam_test['Labels']=y_pred


In [96]:
for i in range(len(tam_test)):
  if tam_test.loc[i,'Labels']==3:
    tam_test.loc[i,'Labels']='unknown_state'
  elif tam_test.loc[i,'Labels']==2:
    tam_test.loc[i,'Labels']='positive'
  elif tam_test.loc[i,'Labels']==1:
    tam_test.loc[i,'Labels']='mixed_feelings'
  else:
    tam_test.loc[i,'Labels']='negative'
tam_test

<ipython-input-96-36c84696fba4>:9: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'negative' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  tam_test.loc[i,'Labels']='negative'


,Id,Text,Labels
0,SA_Ta_01,just likes to reach likes​,negative
1,SA_Ta_02,mas overload. sema vera level,positive
2,SA_Ta_03,she's looks like a dummy (mom ki gudiya),negative
3,SA_Ta_04,எந்த ட்ரைலர் சிறந்தது பிசாசு விருப்பம் சைக்கோ ...,negative
4,SA_Ta_05,adichi under wear oda vituruvean lol,mixed_feelings
...,...,...,...
3447,SA_Ta_3455,je exact ah enakum indha noi irukuji funeral h...,mixed_feelings
3448,SA_Ta_3456,please kaappaanla nadippin nayagan suriya nu p...,unknown_state
3449,SA_Ta_3457,pakka paisa vasol movie. mas movie after naras...,positive
3450,SA_Ta_3458,waiting to se anna mas,positive


In [97]:
tam_test= tam_test.drop(columns=['Text'])


In [98]:
tam_test.to_csv("/content/drive/MyDrive/sentiment analysis/TensorTalk_Tamil_run3.tsv", index=False)

In [99]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

# Initialize the Random Forest Classifier
model = RandomForestClassifier(n_estimators=100, random_state=42)

# Fit the model on the resampled training data
model.fit(X_resampled, y_resampled)

# Make predictions on the development set
y_pred = model.predict(X_dev)
tam_test['Labels']=y_pred



In [100]:
for i in range(len(tam_test)):
  if tam_test.loc[i,'Labels']==3:
    tam_test.loc[i,'Labels']='unknown_state'
  elif tam_test.loc[i,'Labels']==2:
    tam_test.loc[i,'Labels']='positive'
  elif tam_test.loc[i,'Labels']==1:
    tam_test.loc[i,'Labels']='mixed_feelings'
  else:
    tam_test.loc[i,'Labels']='negative'
tam_test

<ipython-input-100-36c84696fba4>:9: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'negative' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  tam_test.loc[i,'Labels']='negative'


,Id,Labels
0,SA_Ta_01,negative
1,SA_Ta_02,positive
2,SA_Ta_03,positive
3,SA_Ta_04,positive
4,SA_Ta_05,unknown_state
...,...,...
3447,SA_Ta_3455,mixed_feelings
3448,SA_Ta_3456,unknown_state
3449,SA_Ta_3457,positive
3450,SA_Ta_3458,positive


In [101]:
tam_test.to_csv("/content/drive/MyDrive/sentiment analysis/TensorTalk_Tamil_run2.csv", index=False)

In [102]:
from sklearn.ensemble import AdaBoostClassifier

# Create an AdaBoost classifier
adaboost_model = AdaBoostClassifier(
    estimator=RandomForestClassifier(max_depth=7),
    n_estimators=100,
    learning_rate=0.1,
    random_state=42
)

# Train the model
adaboost_model.fit(X_resampled, y_resampled)

# Make predictions
y_pred = adaboost_model.predict(X_dev)
tam_test['Labels']=y_pred


In [103]:
for i in range(len(tam_test)):
  if tam_test.loc[i,'Labels']==3:
    tam_test.loc[i,'Labels']='unknown_state'
  elif tam_test.loc[i,'Labels']==2:
    tam_test.loc[i,'Labels']='positive'
  elif tam_test.loc[i,'Labels']==1:
    tam_test.loc[i,'Labels']='mixed_feelings'
  else:
    tam_test.loc[i,'Labels']='negative'
tam_test

<ipython-input-103-36c84696fba4>:3: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'unknown_state' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  tam_test.loc[i,'Labels']='unknown_state'


,Id,Labels
0,SA_Ta_01,unknown_state
1,SA_Ta_02,positive
2,SA_Ta_03,negative
3,SA_Ta_04,mixed_feelings
4,SA_Ta_05,mixed_feelings
...,...,...
3447,SA_Ta_3455,mixed_feelings
3448,SA_Ta_3456,unknown_state
3449,SA_Ta_3457,positive
3450,SA_Ta_3458,positive


In [104]:
tam_test.to_csv("/content/drive/MyDrive/sentiment analysis/TensorTalk_Tamil_run2.tsv", index=False)